In [ ]:
import numpy as np 
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

**The goal of this competition is to predict which of the provided pairs of questions contain two questions with the same meaning. The ground truth is the set of labels that have been supplied by human experts. The ground truth labels are inherently subjective, as the true meaning of sentences can never be known with certainty. Human labeling is also a 'noisy' process, and reasonable people will disagree. As a result, the ground truth labels on this dataset should be taken to be 'informed' but not 100% accurate, and may include incorrect labeling. We believe the labels, on the whole, to represent a reasonable consensus, but this may often not be true on a case by case basis for individual items in the dataset.**

Please note: as an anti-cheating measure, Kaggle has supplemented the test set with computer-generated question pairs. Those rows do not come from Quora, and are not counted in the scoring. All of the questions in the training set are genuine examples from Quora.

Data fields
id - the id of a training set question pair
qid1, qid2 - unique ids of each question (only available in train.csv)
question1, question2 - the full text of each question
is_duplicate - the target variable, set to 1 if question1 and question2 have essentially the same meaning, and 0 otherwise.

**DATASET LINK - https://www.kaggle.com/c/quora-question-pairs/data**

In [ ]:
train=pd.read_csv('/kaggle/input/quora-question-pairs/train.csv.zip')

In [ ]:
train.shape

**We have 404290 observations and 6 features in the dataset.**

In [ ]:
train.head()

In [ ]:
test=pd.read_csv('/kaggle/input/quora-question-pairs/test.csv')

In [ ]:
test.head()

In [ ]:
train[train['is_duplicate']==1].head()

In [ ]:
train.info()

In [ ]:
#dropping null values
train=train.dropna()

In [ ]:
train_list1=list(train['question1'])
train_list2=list(train['question2'])
train_list=train_list1+train_list2

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size=20000
tokenizer=Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_list)

In [ ]:
sequence1=tokenizer.texts_to_sequences(train_list1)
sequence2=tokenizer.texts_to_sequences(train_list2)

In [ ]:
#padding the sequences to a constant size
max_length=100
sequence1=pad_sequences(sequence1,maxlen=max_length,padding='post')
sequence2=pad_sequences(sequence2,maxlen=max_length,padding='post')

In [ ]:
train['seq1']=list(sequence1)

In [ ]:
train['seq2']=list(sequence2)

In [ ]:
train.head()

In [ ]:
labels=np.asarray(train['is_duplicate'])

In [ ]:
#functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Embedding,LSTM,concatenate

**Embedding the LSTM Layer**

In [ ]:
from tensorflow.keras import Input

text_input1=Input(shape=(None,),dtype='int32')
embedding1=Embedding(vocab_size,64)(text_input1)
encoded_text1=LSTM(32)(embedding1)

text_input2=Input(shape=(None,),dtype='int32')
embedding2=Embedding(vocab_size,64)(text_input2)
encoded_text2=LSTM(32)(embedding2)

concatenated=concatenate([encoded_text1,encoded_text2],axis=-1)

output=Dense(64,activation='relu')(concatenated)
output=Dense(1,activation='sigmoid')(output)

**Compiling the model with Adam optimizer and loss as categorical crossentropy and evaluating the results using accuracy.**

In [ ]:
model=Model([text_input1,text_input2],output)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

**Fitting the model**

In [ ]:
hist = model.fit([sequence1,sequence2],labels,epochs = 10,batch_size=128)

**At the end of 10th epoch we are getting an accuracy of 74% and 0.52 loss.**